In [ ]:
%reload_ext autoreload
%autoreload 2

import torch

torch.manual_seed(42)


In [11]:
from data.names_data_source import NamesDataSource, START_TOKEN, END_TOKEN
from learning.names_generator.names_generator_dataset import NamesGeneratorDataset


names_data_source = NamesDataSource.load(
    data_folder="../datasets/names",
    prefix=START_TOKEN,
    suffix=END_TOKEN,
    normalize_unicode=True,
)

names_dataset = NamesGeneratorDataset(names_data_source)


In [ ]:
from learning.names_generator.model import NamesGenerator
import torch
import torch.nn as nn


def generate_name(model: nn.Module, country_idx: int):
    with torch.no_grad():
        # shape: [1, C]
        category_tensor = names_data_source.country_index_to_one_hot(
            country_idx
        ).unsqueeze(0)

        # shape: [1, H]
        hidden_tensor = test_names_generator.init_hidden()

        # shape: [1, V]
        input_tensor = names_data_source.name_to_one_hot(START_TOKEN).squeeze(1)

        output_names = []
        for i in range(50):
            # shape: [1, V]
            output, hidden_tensor = model(category_tensor, input_tensor, hidden_tensor)
            idx = torch.argmax(output, dim=1)
            next_char = names_data_source.index_to_token[int(idx.item())]

            if next_char == END_TOKEN:
                break

            output_names.append(next_char)
            input_tensor = names_data_source.name_to_one_hot(next_char).squeeze(1)

        return "".join(output_names)


test_names_generator = NamesGenerator(
    hidden_size=128,
    num_vocab=names_data_source.num_vocab,
    num_classes=names_data_source.num_classes,
)

generate_name(test_names_generator, 4)
